In [58]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [121]:
from mvf_bto.data_loading import load_data
from mvf_bto.constants import * 
from mvf_bto.models.convolutional_1d import Convolutional1D
# from mvf_bto.preprocessing.discharge import create_discharge_inputs
from mvf_bto.preprocessing.convolutional_charge import create_charge_inputs

from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.metrics import MeanSquaredError
from tensorflow.keras.metrics import MeanAbsoluteError
from scipy.interpolate import interp1d

import numpy as np
import pandas as pd
import plotly
import plotly.graph_objects as go

## Loading Data

In [100]:
# data_path = "/Users/anoushkabhutani/PycharmProjects/10701-mvf-bto/data/2017-05-12_batchdata_updated_struct_errorcorrect.mat"
data_path = "/Users/mac/Desktop/CMU/10701MachineLearning/project/10701-mvf-bto-backup/data/2017-05-12_batchdata_updated_struct_errorcorrect.mat"

In [65]:
data = load_data(file_path=data_path, num_cells=15)

100%|██████████| 15/15 [02:00<00:00,  8.06s/it]


## Preprocessing to create model inputs and targets

In [136]:
train_split = 0.7
test_split = 0.2
forecast_horizon=1
history_window=6
# by default uses validation_split = 1 - (train_split + test_split)

In [137]:
cell_id='b1c8'
policy=data[cell_id]["charge_policy"]
print(policy)

5.4C(40%)-3.6C


In [138]:
print(data.keys())

dict_keys(['b1c0', 'b1c1', 'b1c2', 'b1c3', 'b1c4', 'b1c5', 'b1c6', 'b1c7', 'b1c8', 'b1c9', 'b1c10', 'b1c11', 'b1c12', 'b1c13', 'b1c14'])


In [139]:

datasets = create_charge_inputs(data, train_split, test_split,
                                forecast_horizon=forecast_horizon,
                                history_window=history_window)

['b1c4', 'b1c5', 'b1c13', 'b1c6', 'b1c1', 'b1c2', 'b1c7', 'b1c0', 'b1c10', 'b1c11'] ['b1c8', 'b1c14'] ['b1c12', 'b1c9', 'b1c3']


100%|██████████| 879/879 [00:08<00:00, 105.20it/s]


In [140]:
train_cell_ids = list(datasets['original_train']['Cell'].unique())
test_cell_ids = list(datasets['original_test']['Cell'].unique())
val_cell_ids = list(datasets['original_val']['Cell'].unique())
print(train_cell_ids,test_cell_ids,val_cell_ids)

['b1c4', 'b1c5', 'b1c13', 'b1c6', 'b1c1', 'b1c2', 'b1c7', 'b1c0', 'b1c10', 'b1c11'] ['b1c12', 'b1c9', 'b1c3'] ['b1c8', 'b1c14']


In [141]:
# for cell in train_cell_ids:
#     temp = datasets['original_train'][datasets['original_train'].Cell==cell]
#     print(len(temp.Cycle.unique()))

In [142]:
# print(datasets['original_train']['SOC1'])

In [143]:
# fig=go.Figure()
# cell='b1c3'
# for cycle in range(1,1200,50):
#     temp=datasets['original_train'][(datasets['original_train'].Cell==cell) & (datasets['original_train'].Cycle==cycle)]
#     fig.add_trace(go.Scatter(x=temp['Qc'],
#                             y=temp['temp'],
#                             mode="markers"))
# fig.show()

In [144]:
# fig=go.Figure()
# cell='b1c1'
# for cycle in range(1,1200,50):
#     temp=datasets['original_train'][(datasets['original_train'].Cell==cell) & (datasets['original_train'].Cycle==cycle)]
#     fig.add_trace(go.Scatter(x=temp['Qc'],
#                             y=temp['temp'],
#                             mode="markers"))
# fig.show()

## Show interpolation before training

In [145]:
pallete = plotly.colors.qualitative.Dark24 + plotly.colors.qualitative.T10
pallete = pallete*70000
# pred_df.columns

In [146]:
n_cycles = int(datasets['X_test'].shape[0]/datasets['arrays_per_cycle'])

fig_list = []

for nf_steps in range(0, forecast_horizon*2, 2):
    fig = go.Figure()
    for i in range(0, n_cycles, 40):
        X_cycle = datasets['X_test'][i*datasets['arrays_per_cycle']:(i+1)*datasets['arrays_per_cycle']]
        y_cycle = datasets['y_test'][i*datasets['arrays_per_cycle']:(i+1)*datasets['arrays_per_cycle']]
        # y_hat = model.predict(X_cycle, verbose=0)
        fig.add_trace(go.Scatter(x=[i[nf_steps//2][1] for i in X_cycle[history_window:]], 
                                 y =[i[nf_steps]*(VOLTAGE_MAX-VOLTAGE_MIN)+VOLTAGE_MIN  for i in y_cycle], 
                                 mode="markers+lines", line_color = pallete[i], name=f"data{i}"))
        # fig.add_trace(go.Scatter(x=[i[nf_steps//2][1] for i in X_cycle[history_window:]], 
                                #  y =[i[nf_steps] for i in y_hat], 
                                #  mode="markers", line_color = pallete[i], name= "Prediction"))
    fig.update_xaxes(title="Normalized Capacity [Ah]")
    fig.update_yaxes(title="Normalized Voltage [V]")
    fig.update_layout(title=f"Voltage Prediction at Forecast Horizon {int(nf_steps//2)+1}")
    fig_list.append(fig)

In [147]:
fig_list[0]

## Train Model

In [148]:
window_length = datasets["X_train"].shape[1]
n_features = datasets['n_features']
input_shape = (window_length, n_features)
output_dimension = datasets["y_train"].shape[-1]
print(window_length, n_features, input_shape, output_dimension)
print(datasets["y_train"].shape)
print(datasets["X_train"].shape)

6 6 (6, 6) 2
(373074, 2)
(373074, 6, 6)


In [149]:
from tensorflow import keras

In [150]:
model = Convolutional1D(
    input_shape=input_shape, 
    n_outputs=output_dimension,
)


In [151]:
lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.01,
    decay_steps=1000,
    decay_rate=0.96)
optimizer = keras.optimizers.Adam(learning_rate=lr_schedule)

model.compile(
    optimizer=optimizer,
    loss='mae',
    metrics=[MeanAbsoluteError()]
)

In [152]:
es = EarlyStopping(
    monitor="val_mean_absolute_error",
    min_delta=0,
    patience=40,
    verbose=1,
    mode="min",
    restore_best_weights=True,
)

history = model.fit(
    datasets["X_train"],
    datasets["y_train"],
    validation_data=(datasets["X_val"], datasets["y_val"]),
    epochs=300,
    callbacks=[es],
    batch_size=128,
    shuffle=False,
    verbose=1,

)

Epoch 1/300
2915/2915 [==============================] - 10s 3ms/step - loss: 0.0880 - mean_absolute_error: 0.0880 - val_loss: 0.0874 - val_mean_absolute_error: 0.0874
Epoch 2/300
2915/2915 [==============================] - 8s 3ms/step - loss: 0.0831 - mean_absolute_error: 0.0831 - val_loss: 0.0920 - val_mean_absolute_error: 0.0920
Epoch 3/300
2915/2915 [==============================] - 9s 3ms/step - loss: 0.0804 - mean_absolute_error: 0.0804 - val_loss: 0.0879 - val_mean_absolute_error: 0.0879
Epoch 4/300
2915/2915 [==============================] - 8s 3ms/step - loss: 0.0793 - mean_absolute_error: 0.0793 - val_loss: 0.0764 - val_mean_absolute_error: 0.0764
Epoch 5/300
2915/2915 [==============================] - 8s 3ms/step - loss: 0.0771 - mean_absolute_error: 0.0771 - val_loss: 0.0828 - val_mean_absolute_error: 0.0828
Epoch 6/300
2915/2915 [==============================] - 8s 3ms/step - loss: 0.0762 - mean_absolute_error: 0.0762 - val_loss: 0.0918 - val_mean_absolute_error: 0.09

In [99]:
model.save("/Users/mac/Desktop/CMU/10701MachineLearning/project/10701-mvf-bto/pre_train/cnn_charge/Ch10p1ID45noSOC")

INFO:tensorflow:Assets written to: /Users/mac/Desktop/CMU/10701MachineLearning/project/10701-mvf-bto/pre_train/cnn_charge/Ch10p1ID45noSOC/assets


INFO:tensorflow:Assets written to: /Users/mac/Desktop/CMU/10701MachineLearning/project/10701-mvf-bto/pre_train/cnn_charge/Ch10p1ID45noSOC/assets


## Parity Plot of Training Error

In [153]:
pred_df = pd.DataFrame(model.predict(datasets["X_test"], verbose=0))
test_df = pd.DataFrame(datasets["y_test"])
column_names = []
for i in range(len(test_df.columns)//2):
    column_names.append(f"Voltage (FH = {i+1})")
    column_names.append(f"Temperature (FH = {i+1})")
pred_df.columns =  column_names
test_df.columns =  column_names

In [154]:
skip = 30
parity_plot_list = []
for column in test_df.columns:
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=[0, 1], y=[0, 1], showlegend=False, mode="lines"))
    fig.add_trace(go.Scatter(x = pred_df[column][::skip], showlegend=False,  y=test_df[column][::skip], mode="markers"))
    fig.update_yaxes(title=f"Normalized Target {column}")
    fig.update_xaxes(title=f"Normalized Prediction {column}")
    fig.update_layout(template="simple_white")
    parity_plot_list.append(fig)

In [155]:
parity_plot_list[0]

## True vs Predicted Traces (Test Set)

In [116]:
cycles = [int(datasets["X_test"][i][-1][-1]*MAX_CYCLE) for i in range(len(datasets["X_test"]))]
pred_df = pd.DataFrame(model.predict(datasets["X_test"], verbose=1))
test_df = pd.DataFrame(datasets["y_test"])
column_names = []
for i in range(len(test_df.columns)//2):
    column_names.append(f"Voltage (FH = {i+1})")
    column_names.append(f"Temperature (FH = {i+1})")
pred_df.columns =  column_names
test_df.columns =  column_names
test_df['Cycle']= cycles
pred_df['Cycle']=cycles
cycles_plot = pred_df['Cycle'].unique()[::30]
pred_df['Q'] = [datasets["X_test"][i][0][1]*1.6 for i in range(0,len(datasets["X_test"]))]
test_df['Q'] = [datasets["X_test"][i][0][1]*1.6 for i in range(0,len(datasets["X_test"]))]
test_cell_ids = datasets['original_test'].Cell.unique()

1882/1882 [==============================] - 4s 2ms/step


In [117]:
pallete = plotly.colors.qualitative.Dark24 + plotly.colors.qualitative.T10
pallete = pallete*70000
pred_df.columns

Index(['Voltage (FH = 1)', 'Temperature (FH = 1)', 'Cycle', 'Q'], dtype='object')

In [118]:
split_indices = [0,]
# split_indices.extend([i[0] for i in np.where(np.diff(pred_df.Cycle)<0)])
split_indices.append(len(pred_df))
cell_wise_pred_dfs = [pred_df.iloc[split_indices[n]:split_indices[n+1]] for n in range(len(split_indices)-1)]
cell_wise_test_dfs = [test_df.iloc[split_indices[n]:split_indices[n+1]] for n in range(len(split_indices)-1)]

In [119]:
n_cycles = int(datasets['X_test'].shape[0]/datasets['arrays_per_cycle'])

fig_list = []

for nf_steps in range(0, forecast_horizon*2, 2):
    fig = go.Figure()
    for i in range(0, n_cycles, 500):
        X_cycle = datasets['X_test'][i*datasets['arrays_per_cycle']:(i+1)*datasets['arrays_per_cycle']]
        y_cycle = datasets['y_test'][i*datasets['arrays_per_cycle']:(i+1)*datasets['arrays_per_cycle']]
        y_hat = model.predict(X_cycle, verbose=0)
        fig.add_trace(go.Scatter(x=[i[nf_steps//2][1] for i in X_cycle[history_window:]], 
                                 y =[i[nf_steps]  for i in y_cycle], 
                                 mode="markers+lines", line_color = pallete[i], name="Data"))
        fig.add_trace(go.Scatter(x=[i[nf_steps//2][1] for i in X_cycle[history_window:]], 
                                 y =[i[nf_steps] for i in y_hat], 
                                 mode="markers", line_color = pallete[i+1], name= "Prediction"))
    fig.update_xaxes(title="Normalized Capacity [Ah]")
    fig.update_yaxes(title="Normalized Voltage [V]")
    fig.update_layout(title=f"Voltage Prediction at Forecast Horizon {int(nf_steps//2)+1}")
    fig_list.append(fig)

In [120]:
fig_list[0]

In [39]:
model.summary()

Model: "convolutional1d"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             multiple                  3648      
                                                                 
 max_pooling1d (MaxPooling1D  multiple                 0         
 )                                                               
                                                                 
 flatten (Flatten)           multiple                  0         
                                                                 
 dense (Dense)               multiple                  8320      
                                                                 
 dense_1 (Dense)             multiple                  6450      
                                                                 
 dense_2 (Dense)             multiple                  102       
                                                   

In [40]:
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=np.linspace(1, 160),
        y=history.history["loss"],
        showlegend=False,
        mode="markers+lines",
    )
)
fig.update_xaxes(title="Epochs")
fig.update_yaxes(title="Loss (MSE)")